# ManyCores exercises

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import pathlib
from matplotlib import rcParams
import itertools
%matplotlib inline

from IPython.display import display

In [ ]:
def csv_parse(path, header=False, sep=";"):
    if header:
        df = pd.read_csv(path, sep=sep, names=header)
    else:
        df = pd.read_csv(path, sep=sep)
    #display(df.head())
    return df

def load_glob(glob, concat=True):
    liste = [pd.read_csv(file) for file in glob]
    name = [file for file in glob]
    repr(name)
    if concat:
        return pd.concat(liste)
    return liste, name

In [ ]:
def transform_to_ms(df, cols):
    return df[cols]*1e3

def bandwidth(df, cols):
    for x in cols:
        df["bw_"+x] = 2 * df["N"]*df["N"]* 8 / df[x] * 1e-9
    return df

# Ex10.1

In [ ]:
# Get the data
cwd = pathlib.Path(os.getcwd())
data_subdir = cwd.parent / "data"
plot_dir = cwd / "out"
data_subdir, plot_dir

In [ ]:
df_hip = csv_parse(data_subdir / "ph_data_hip.csv")
df_cuda = csv_parse(data_subdir / "ph_data_cuda.csv")
display(df_hip)
df_cuda 

In [ ]:
df_speedup = df_hip.copy()
df_speedup["speedup"] = df_speedup["runtime"] / df_cuda["runtime"]
df_speedup

In [ ]:
def print_styling():

    plt.rc('figure', figsize=(12,8)) 
    plt.style.use("seaborn")
    #sns.set_palette("Paired")
    SMALL_SIZE = 15
    MEDIUM_SIZE = 18
    BIGGER_SIZE = 26

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('lines', linewidth=2)

    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
print_styling()

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(12,8))
print_styling()
axs = axs.ravel()
axL = axs[0]
axR = axs[1]

logx, logy = True, True
if (logx and logy):
    plot_format = "loglog"
if (logx and not logy):
    plot_format = "logx"
if (not logx and logy):
    plot_format = "logy"
else:
    plot_format = "linear"

df_hip.plot(x="N", y=["runtime"], ax=axL, logx=logx, logy=logy, marker="s", ms=10, ls="--")
df_cuda.plot(x="N", y=["runtime"], ax=axL, logx=logx, logy=logy, marker="p",ls="--")
df_speedup.plot(x="N", y=["speedup"], ax=axR, logx=logx, logy=False, marker="o")

axL.set_ylabel("runtime [s]")
axR.set_ylabel("speedup [hip/cuda]")

axL.set_title("Runtime")
axR.set_title("Speedup (HIP/CUDA)")

axL.legend(["HIP", "CUDA"])
axR.legend(["HIP / CUDA"])

plt.suptitle("HIP (CUDA) vs pure CUDA")
plt.tight_layout()
fig.savefig(plot_dir / "hip_vs_cuda.pdf", bbox_inches="tight", pad_inches=0.2)

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(12,8))
print_styling()
axs = axs.ravel()
axL = axs[0]
axR = axs[1]

logx, logy = True, True
if (logx and logy):
    plot_format = "loglog"
if (logx and not logy):
    plot_format = "logx"
if (not logx and logy):
    plot_format = "logy"
else:
    plot_format = "linear"

df_hip.plot(x="N", y=["iterations"], ax=axL, logx=logx, logy=logy, marker="s", ms=10, ls="")
df_cuda.plot(x="N", y=["iterations"], ax=axL, logx=logx, logy=logy, marker="p", ls="--")
df_hip.plot(x="N", y=["residual"], ax=axR, logx=logx, logy=logy, marker="s", ms=10, ls="")
df_cuda.plot(x="N", y=["residual"], ax=axR, logx=logx, logy=logy, marker="p", ls="--")

axL.set_ylabel("iterations # ")
axR.set_ylabel("relative residual")

axL.set_title("Iterations needed")
axR.set_title("Residual")

axL.legend(["HIP", "CUDA"])
axR.legend(["HIP", "CUDA"])

plt.suptitle("HIP (CUDA) vs pure CUDA")
plt.tight_layout()
fig.savefig(plot_dir / "hip_vs_cuda_iter+res.pdf", bbox_inches="tight", pad_inches=0.2)

# Task 2: Transpose

In [ ]:
df_host = csv_parse(data_subdir / "ph_data_host.csv")
df_sycl = csv_parse(data_subdir / "ph_data_sycl.csv")
df_host["runtime/iter"] = df_host["runtime"] / df_host["iterations"]
df_sycl["runtime/iter"] = df_sycl["runtime"] / df_sycl["iterations"]
display(df_host)
df_sycl 

In [ ]:
print(df_host.to_latex())

In [ ]:
print(df_sycl.to_latex())

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(12,8))
print_styling()
axs = axs.ravel()
axL = axs[0]
axR = axs[1]

logx, logy = True, True
if (logx and logy):
    plot_format = "loglog"
if (logx and not logy):
    plot_format = "logx"
if (not logx and logy):
    plot_format = "logy"
else:
    plot_format = "linear"

df_host.plot(x="N", y=["runtime"], ax=axL, logx=logx, logy=logy, marker="s", ms=10, ls="--")
df_sycl.plot(x="N", y=["runtime"], ax=axL, logx=logx, logy=logy, marker="p",ls="--")
df_host.plot(x="N", y=["runtime/iter"], ax=axR, logx=logx, logy=logy, marker="s", ms=10, ls="--")
df_sycl.plot(x="N", y=["runtime/iter"], ax=axR, logx=logx, logy=logy, marker="p",ls="--")


axL.set_ylabel("runtime [s]")

axL.set_title("Runtime")
axR.set_title("Runtime per iteration")

axL.legend(["Host", "SyCL"])
axR.legend(["Host", "SyCL"])

plt.suptitle("HIP (CUDA) vs pure CUDA")
plt.tight_layout()
fig.savefig(plot_dir / "host_vs_sycl.pdf", bbox_inches="tight", pad_inches=0.2)